In [21]:
import tkinter as tk
from tkinter import ttk
from aslm.view.custom_widgets.validation import ValidatedMixin
from decimal import Decimal, InvalidOperation
from aslm.view.custom_widgets.hover import hover


class ValidatedMixin:
    """
    #### Adds validation functionality to an input widget
    """
    # error_var can be passed a var for error message, if not class creates its own
    def __init__(self, *args, error_var=None, **kwargs):
        self.error = error_var or tk.StringVar()
        super().__init__(*args, **kwargs) # Calls base class that is mixed in with this class

        # Validation setup
        validcmd = self.register(self._validate)
        invalidcmd = self.register(self._invalid)

        # Tkinter widget validation setup
        self.config(
            validate='all', # Includes all validation events keystroke and focus
            validatecommand=(validcmd, '%P', '%s', '%S', '%V', '%i', '%d'), # pass in all sub codes/data
            invalidcommand=(invalidcmd, '%P', '%s', '%S', '%V', '%i', '%d')
        )
        
        self.hover = hover(self, text=None, type="free")

    # Error handler - where you can customize color or what happens to widget
    def _toggle_error(self, on=False):
        self.config(foreground=('red' if on else 'black')) # Changes text to red on error

    # Validation, args are the sub codes. This just sets up validation then runs based on event type
    def _validate(self, proposed, current, char, event, index, action):
        self._toggle_error(False) # Error is off
        self.error.set("") # No error to start
        valid=True # Again true means no error
        if event == 'focusout': # Leaving widget
            valid = self._focusout_validate(event=event)
        elif event == 'key': # Keystroke into widget
            valid = self._key_validate(proposed=proposed, current=current, char=char, event=event, index=index, action=action)
        return valid

    # Sub validation functions to be overridden by specific widgets
    def _focusout_validate(self, **kwargs): # **kwargs lets us just specify needed keywords or get args from **kwargs(double pointer ie array) rather than getting args in right order
        return True
    
    def _key_validate(self, **kwargs):
        return True

    # Invalid
    def _invalid(self, proposed, current, char, event, index, action):
        if event == 'focusout':
            self._focusout_invalid(event=event)
        elif event == 'key':
            self._key_invalid(proposed=proposed, current=current, char=char, event=event, index=index, action=action)
        
    def _focusout_invalid(self, **kwargs):
        self._toggle_error(True)
    
    def _key_invalid(self, **kwargs):
        pass
    
    # Allows a manual check on entered values to be used whenever needed
    def trigger_focusout_validation(self):
        valid = self._validate('', '', '', 'focusout', '', '')
        if not valid:
            self._focusout_invalid(event='focusout')
        return valid

In [22]:
class ValidatedEntry(ValidatedMixin, ttk.Entry):
    def __init__(self, *args, precision=0, required=False, min_var=None, max_var=None, focus_update_var=None, min='-Infinity', max='Infinity', **kwargs):
        super().__init__(*args, **kwargs)
        self.resolution = Decimal(precision) # Number for precision given on creation
        self.precision = (self.resolution.normalize().as_tuple().exponent) # Precision of number as exponent
        self.variable = kwargs.get('textvariable') or tk.StringVar
        self.min = min
        self.max = max
        self.required = required
        self.undo_history = []
        self.redo_history = []
        
    # Dynamic range checker
        # if min_var:
        #     self.min_var = min_var
        #     self.min_var.trace_add('w', self._set_minimum)
        # if max_var:
        #     self.max_var = max_var
        #     self.max_var.trace_add('w', self._set_maximum)
        # self.focus_update_var = focus_update_var
        # self.bind('<FocusOut>', self._set_focus_update_var)

    # Binding key press
        # self.bind('<Key>', self.key_print)

    # Update history
        self.bind("<FocusOut>", self.add_history)

    # Undo/Redo
        # self.bind("z", self.undo)
        # self.bind("y", self.redo)

    
    # All widgets that want to use undo redo will need this function and the undo_history, redo_history lists
    def add_history(self, event):
        print("Add history process started")
        if event.type == '10':
            print("FocusOut event caught in history")
            if self.get() != "":
                print("Not a blank focusout")
                if not self.undo_history or self.get() != self.undo_history[-1]:
                    print("No repeat entry or undo history is empty")
                    self.undo_history.append(self.get())
                    print("Undo history list: ", event.widget.undo_history)


    # def undo(event):
    #     if event.state == 4:
    #         if len(event.widget.undo_history) > 1:
    #             event.widget.delete(0, tk.END)
    #             event.widget.insert(tk.END, event.widget.undo_history[-2])
    #             event.widget.redo_history.append(event.widget.undo_history.pop())
    #         elif len(event.widget.undo_history) == 1:
    #             event.widget.delete(0, tk.END)
    #             event.widget.redo_history.append(event.widget.undo_history.pop())
            
    
    # def redo(event):
    #     if event.state == 4:
    #         if event.widget.redo_history:
    #             event.widget.delete(0, tk.END)
    #             event.widget.insert(tk.END, event.widget.redo_history[-1])
    #             event.widget.undo_history.append(event.widget.redo_history.pop())

    # def key_print(self, event):
    #     print(event.state)
    #     print(event.keysym)
    #     print(event.keycode)
    #     print(event)

    def _get_precision(self):
        nums_after = self.resolution.find('.')
        
        return (-1) * len(self.resolution[nums_after + 1:])

    def _key_validate(self, char, index, current, proposed, action, **kwargs):

        valid = True
        min_val = self.min
        max_val = self.max

        # check if there are range limits
        # TODO: I add a line here, please make sure is it okay?
        if min_val == '-Infinity' or max_val == 'Infinity':
            return True

        no_negative = int(min_val) >= 0
        no_decimal = self.precision >= 0


        # Allow deletion
        if action == '0':
            return True

        # Testing keystroke for validity
        # Filter out obviously bad keystrokes
        if any([
            (char not in ('-1234567890.')),
            (char == '-' and (no_negative or index != '0')),
            (char == '.' and (no_decimal or '.' in current))
        ]):
            return False

        # Proposed is either a Decimal, '-', '.', or '-.' need one final check for '-' and '.' 
        if proposed in '-.':
            return True

        # Proposed value is a Decimal, so convert and check further
        proposed = Decimal(proposed)
        proposed_precision = proposed.as_tuple().exponent
        if any([
            (proposed > int(max_val)),
            (proposed_precision < self.precision)
        ]):
            return False
        
        return valid

    

    # If entry widget is empty set the error string and return False TODO add hover bubble with error message
    def _focusout_validate(self, event):
        valid = True
        value = self.get()
        max_val = self.max
        min_val = self.min
        # Check for error upon leaving widget
        # TODO: I add some lines here, please make sure are they okay?
        if value.strip() == '' and self.required:
            self.error.set('A value is required')
            return False
        else:
            self.error.set('')
        
        # check if there are range limits
        if min_val == '-Infinity' or max_val == 'Infinity':
            return True

        try:
            value = Decimal(value)
        except InvalidOperation:
            self.error.set('Invalid number string: {}'.format(value))
            return False
        
        # Checking if greater than minimum
        if value < int(min_val):
            self.error.set('Value is too low (min {})'.format(min_val))
            valid = False

        # Checking if less than max
        if value > int(max_val):
            self.error.set('Value is too high (max {})'.format(max_val))

        

        return valid
    
    # Gets current value of widget and if focus_update_var is present it sets it to the same value
    def _set_focus_update_var(self, event):
        value = self.get()
        if self.focus_update_var and not self.error.get():
            self.focus_update_var.set(value)
    
    # Update minimum based on given variable
    def _set_minimum(self, *args):
        current = self.get()
        try:
            new_min = self.min_var.get()
            self.min = new_min
            print(self.max)
            logger.info(self.max)
        except (tk.TclError, ValueError):
            pass
        if not current:
            self.delete(0, tk.END)
        else:
            self.variable.set(current)
        self.trigger_focusout_validation() # Revalidate with the new minimum
    
    # Update maximum based on given variable
    def _set_maximum(self, *args):
        current = self.get()
        try:
            new_max = self.max_var.get()
            self.max = new_max
        except (tk.TclError, ValueError):
            pass
        if not current:
            self.delete(0, tk.END)
        else:
            self.variable.set(current)
        self.trigger_focusout_validation() # Revalidate with the new maximum
        
    def _toggle_error(self, on=False):
        super()._toggle_error(on)
        if on:
            self.hover.seterror(self.error.get())
        else:
            self.hover.hidetip()

In [23]:
def undo(event):
        print("Undo process started")
        if isinstance(event.widget, ValidatedEntry): #Add all widgets that you want to be able to undo here
            widget = event.widget
            print("Entry being tested: ", widget)
            print("State of widget: ", widget.state)
            print("Undo history of widget: ", widget.undo_history)
            if event.state == 4:
                if len(widget.undo_history) > 1:
                    print("Undo really starting now")
                    widget.delete(0, tk.END)
                    print("Widget delete")
                    widget.insert(tk.END, widget.undo_history[-2]) # Should this be widget.variable.set()
                    print("Widget insert: ", widget.undo_history[-2])
                    widget.redo_history.append(widget.undo_history.pop())
                    print("Adding to redo history: ", widget.redo_history)
                    print("New undo history: ", widget.undo_history)
                if len(widget.undo_history) == 1:
                    # widget.delete(0, tk.END)
                    print("Widget delete bc less than 1")
                    widget.redo_history.append(widget.undo_history.pop())
                    print("Adding to redo history: ", widget.redo_history)
                    print("New undo history: ", widget.undo_history)



def redo(event):
    print("Redo process started")
    if isinstance(event.widget, ValidatedEntry):
        widget = event.widget
        print("Entry being tested: ", widget)
        print("State of widget: ", widget.state)
        print("Redo history of widget: ", widget.redo_history)
        if event.state == 4:
            if widget.redo_history:
                if not widget.undo_history:
                    widget.undo_history.append(widget.redo_history.pop())
                    if not widget.redo_history:
                        return
                print("Redo actually starting")
                widget.delete(0, tk.END)
                print("Widget delete in redo")
                widget.insert(tk.END, widget.redo_history[-1])
                print("Widget insert: ", widget.redo_history[-1])
                widget.undo_history.append(widget.redo_history.pop())
                print("Adding to undo history: ", widget.undo_history)
                print("New redo history: ", widget.redo_history)
                

In [24]:
root = tk.Tk()
entry = ValidatedEntry(root)
entry1 = ttk.Entry(root)
entry.pack()
entry1.pack()
root.bind_all("z", undo)
root.bind_all("y", redo)
root.mainloop()

Add history process started
FocusOut event caught in history
Not a blank focusout
No repeat entry or undo history is empty
Undo history list:  ['hello']
Add history process started
FocusOut event caught in history
Not a blank focusout
No repeat entry or undo history is empty
Undo history list:  ['hello', 'there']
Add history process started
FocusOut event caught in history
Not a blank focusout
No repeat entry or undo history is empty
Undo history list:  ['hello', 'there', 'again']
Undo process started
Entry being tested:  .!validatedentry
State of widget:  <bound method Widget.state of <__main__.ValidatedEntry object .!validatedentry>>
Undo history of widget:  ['hello', 'there', 'again']
Undo really starting now
Widget delete
Widget insert:  there
Adding to redo history:  ['again']
New undo history:  ['hello', 'there']
Undo process started
Entry being tested:  .!validatedentry
State of widget:  <bound method Widget.state of <__main__.ValidatedEntry object .!validatedentry>>
Undo histor